In [4]:
# Loading all yfinance_data
import pandas as pd
import glob
import os

def load_csv_files_from_folder(folder_path):

    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    loaded_dataframes = []

    if not csv_files:
        # Minimal report: only if no files are found at all
        print(f"No CSV files found in '{folder_path}'.")
        return loaded_dataframes

    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            loaded_dataframes.append(df)
        except Exception:
            # Minimal error handling: simply skip the file and do not report details
            pass # No action/report as requested ("just do only that")

    return loaded_dataframes

# --- Execution ---
yfinance_data_folder = '../data/yfinance_data/' 

# Load all CSVs into a list of DataFrames
all_yfinance_dfs = load_csv_files_from_folder(yfinance_data_folder)


print(f"Successfully loaded {len(all_yfinance_dfs)} DataFrames.")
if all_yfinance_dfs:
    print("\nHead of the first loaded DataFrame:")
    print(all_yfinance_dfs[0].head())

# Loading news data
try:
    # Attempt to load the dataset from 'raw_analyst_ratings.csv'
    df = pd.read_csv('../data/raw_analyst_ratings.csv/raw_analyst_ratings.csv')
    print("\n\n\n Dataset loaded successfully. Displaying the first 5 rows and info:")
    print(df.head())
    print("\n")
    df.info()
except FileNotFoundError:
    # If the file isn't found, print an error message and exit the script
    print("Error: 'raw_analyst_ratings.csv' not found.")
    exit() # Terminate execution as we can't proceed without the data
except Exception as e:
    # Catch any other potential errors during file loading (e.g., corrupted CSV)
    print(f"An unexpected error occurred while loading the dataset: {e}")

Successfully loaded 7 DataFrames.

Head of the first loaded DataFrame:
         Date      Open      High       Low     Close  Adj Close      Volume  \
0  1986-03-13  0.088542  0.101563  0.088542  0.097222   0.059946  1031788800   
1  1986-03-14  0.097222  0.102431  0.097222  0.100694   0.062087   308160000   
2  1986-03-17  0.100694  0.103299  0.100694  0.102431   0.063158   133171200   
3  1986-03-18  0.102431  0.103299  0.098958  0.099826   0.061552    67766400   
4  1986-03-19  0.099826  0.100694  0.097222  0.098090   0.060482    47894400   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  



 Dataset loaded successfully. Displaying the first 5 rows and info:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   